# Main

> Entry point for the app


In [1]:
#| default_exp main

In [1]:
#| export
from fasthtml.common import *
from monsterui.all import *
from fasthtml.oauth import *


In [2]:
#| export
try:
    from productivity_app.social_media import ar as ar_social
    from productivity_app.todo import ar as ar_todo
    from productivity_app.utils import *
except Exception as e:
    raise e

In [ ]:
#| expor

In [18]:
#| export
app, rt = fast_app(hdrs = Theme.blue.headers(), body_wrap=layout)

In [13]:
#| export
@rt
def index(): return Grid(*[A(Card(H3(k)),href=v) for k,v in links.items()])

In [14]:
#| export
ar_social.to_app(app)
ar_todo.to_app(app)

In [15]:
#| export
redir_path = '/redirect'
skip = ('/login', redir_path, r'/.*\.(png|jpg|ico|css|js)', )

class Auth(OAuth):
    def get_auth(self, info, ident, session, state):
        email = info.email or ''
        session['user_name'] = email
        if info.email_verified and email.split('@')[-1] == 'answer.ai': return RedirectResponse('/', status_code=303)
        
client = GoogleAppClient(os.environ.get('GOOGLE_CLIENT_ID'), os.environ.get('GOOGLE_SECRET'))
oauth = Auth(app, client, skip=skip, redir_path=redir_path)

@rt
def login(req):
    return Container(Card(
        CardTitle("Login"),
        P("For internal use only", cls=TextPresets.muted_sm),
        A('Log in', href=oauth.login_link(req), cls='uk-button'+ButtonT.primary)))

@rt
def logout(sess):
    del sess["user_name"]
    return oauth.logout(sess)

In [16]:
#| export
serve()